In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import ArrayType
import sys

spark_session = SparkSession\
        .builder\
        .master("local") \
        .appName("Question_B2")\
        .getOrCreate()

spark_context = spark_session.sparkContext
sqlContext = SQLContext(spark_context)

In [2]:
df = spark_session.read.csv('/home/ubuntu/LDSA/lab2/gap_data.csv', header="true", inferSchema="true", multiLine="true", escape="\"")
dfAppendix = spark_session.read.csv('/home/ubuntu/LDSA/lab2/sicCodes.csv', header="true", inferSchema="true", multiLine="true", escape="\"")
df.createOrReplaceTempView("gap_data")

In [12]:
df.select("SicCodes").show()

+--------+
|SicCodes|
+--------+
|   85310|
|   47730|
|   61900|
|   78300|
|   93110|
|   56290|
|   10120|
|   10120|
|   10110|
|   82200|
|   56103|
|   52290|
|   78200|
|   81210|
|   52290|
|   82990|
|   70229|
|   70100|
|   56290|
|   33150|
+--------+
only showing top 20 rows



In [11]:
df.printSchema()

root
 |-- EmployerName: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- CompanyNumber: string (nullable = true)
 |-- SicCodes: string (nullable = true)
 |-- DiffMeanHourlyPercent: double (nullable = true)
 |-- DiffMedianHourlyPercent: double (nullable = true)
 |-- DiffMeanBonusPercent: double (nullable = true)
 |-- DiffMedianBonusPercent: double (nullable = true)
 |-- MaleBonusPercent: double (nullable = true)
 |-- FemaleBonusPercent: double (nullable = true)
 |-- MaleLowerQuartile: double (nullable = true)
 |-- FemaleLowerQuartile: double (nullable = true)
 |-- MaleLowerMiddleQuartile: double (nullable = true)
 |-- FemaleLowerMiddleQuartile: double (nullable = true)
 |-- MaleUpperMiddleQuartile: double (nullable = true)
 |-- FemaleUpperMiddleQuartile: double (nullable = true)
 |-- MaleTopQuartile: double (nullable = true)
 |-- FemaleTopQuartile: double (nullable = true)
 |-- CompanyLinkToGPGInfo: string (nullable = true)
 |-- ResponsiblePerson: string (nullable = 

In [3]:
dfAppendix.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Min: integer (nullable = true)
 |-- Max: integer (nullable = true)



In [10]:
dfAppendix.show()

+---+--------------------+---+---+
|_c0|              Sector|Min|Max|
+---+--------------------+---+---+
|  A|AGRICULTURE FORES...|  1|  3|
|  B|MINING AND QUARRYING|  5|  9|
|  C|       MANUFACTURING| 10| 33|
|  D|ELECTRICITY GAS S...| 35| 35|
|  E|WATER SUPPLY SEWE...| 36| 39|
|  F|        CONSTRUCTION| 41| 43|
|  G|WHOLESALE AND RET...| 45| 47|
|  H|TRANSPORTATION AN...| 49| 53|
|  I|ACCOMMODATION AND...| 55| 56|
|  J|INFORMATION AND C...| 58| 63|
|  K|FINANCIAL AND INS...| 64| 66|
|  L|REAL ESTATE ACTIV...| 68| 68|
|  M|PROFESSIONAL SCIE...| 69| 75|
|  N|ADMINISTRATIVE AN...| 77| 82|
|  O|PUBLIC ADMINISTRA...| 84| 84|
|  P|           EDUCATION| 85| 85|
|  Q|HUMAN HEALTH AND ...| 86| 88|
|  R|ARTS ENTERTAINMEN...| 90| 93|
|  S|OTHER SERVICE ACT...| 94| 96|
|  T|ACTIVITIES OF HOU...| 97| 98|
+---+--------------------+---+---+
only showing top 20 rows



In [15]:
dfAppendix.select(array('Min','Max').alias("min_max")).show()

+--------+
| min_max|
+--------+
|  [1, 3]|
|  [5, 9]|
|[10, 33]|
|[35, 35]|
|[36, 39]|
|[41, 43]|
|[45, 47]|
|[49, 53]|
|[55, 56]|
|[58, 63]|
|[64, 66]|
|[68, 68]|
|[69, 75]|
|[77, 82]|
|[84, 84]|
|[85, 85]|
|[86, 88]|
|[90, 93]|
|[94, 96]|
|[97, 98]|
+--------+
only showing top 20 rows



In [ ]:
dfAppendix.select('Min','Max')

In [ ]:
def to_array(x):
    min = x[0]
    max = x[1]
    return [x[0],x[1]]

In [ ]:
merge_cols = udf(lambda arr: to_array(arr), ArrayType(IntegerType()))

In [ ]:
dfAppendix.withColumn('Min_Max', merge_cols(dfAppendix.select('Min', 'Max'))).show()

In [ ]:
#Get_min_max_udf = udf(lambda y: y.split(',')[2], IntegerType())
#df_with_min_max = dfAppendix.withColumn("_c0",Get_min_max_udf("dfAppendix"))
sum_cols = udf(lambda arr: sum(arr), IntegerType())

In [ ]:
sqlContext.createDataFrame([(101, 1, 16, 8)], ['ID', 'A', 'B', 'C'])\
.withColumn('Result', sum_cols(array('A', 'B', 'C'))).show()

In [13]:
df = df.withColumn('SicCodes', df.SicCodes.substr(1, 2))

In [ ]:
df.drop('age').collect()

In [7]:
df = df.filter(df["SicCodes"] != 1)

In [19]:
composite = df.join(dfAppendix,(df['SicCodes']<= dfAppendix['Max'])&(df['SicCodes']>= dfAppendix['Min']))\
.select("sector",'EmployerName','SicCodes','DiffMeanHourlyPercent')
composite.show()

+--------------------+--------------------+--------+---------------------+
|              sector|        EmployerName|SicCodes|DiffMeanHourlyPercent|
+--------------------+--------------------+--------+---------------------+
|           EDUCATION|"Bryanston School...|      85|                 18.0|
|WHOLESALE AND RET...|"RED BAND" CHEMIC...|      47|                  2.3|
|INFORMATION AND C...|         118 LIMITED|      61|                  1.7|
|ADMINISTRATIVE AN...|   123 EMPLOYEES LTD|      78|                 41.0|
|ARTS ENTERTAINMEN...|        1610 LIMITED|      93|                -22.0|
|ACCOMMODATION AND...| 1STOP HALAL LIMITED|      56|                 11.9|
|       MANUFACTURING|2 SISTERS FOOD GR...|      10|                 15.3|
|       MANUFACTURING|2 SISTERS POULTRY...|      10|                  4.9|
|       MANUFACTURING|2 SISTERS RED MEA...|      10|                 14.5|
|ADMINISTRATIVE AN...|2 Touch Bpo Servi...|      82|                 -0.2|
|ACCOMMODATION AND...|23.

In [20]:
composite.createOrReplaceTempView("composite")

In [28]:
sqlContext.sql("SELECT sector, AVG(DiffMeanHourlyPercent) FROM composite GROUP BY sector ORDER BY AVG(DiffMeanHourlyPercent) ASC").show()

+--------------------+--------------------------+
|              sector|avg(DiffMeanHourlyPercent)|
+--------------------+--------------------------+
|ACTIVITIES OF HOU...|         3.133333333333333|
|HUMAN HEALTH AND ...|         6.708316430020282|
|ACCOMMODATION AND...|        7.7013015184381715|
|WATER SUPPLY SEWE...|         8.993750000000004|
|TRANSPORTATION AN...|         9.396632996633002|
|PUBLIC ADMINISTRA...|                       9.7|
|ACTIVITIES OF EXT...|         9.944444444444445|
|ADMINISTRATIVE AN...|        11.528625592417066|
|           EDUCATION|         11.65329949238578|
|OTHER SERVICE ACT...|         12.46292682926829|
|       MANUFACTURING|        12.979506641366225|
|WHOLESALE AND RET...|        13.858345428156746|
|ELECTRICITY GAS S...|        14.176470588235293|
|REAL ESTATE ACTIV...|         15.22525252525252|
|PROFESSIONAL SCIE...|         18.81478537360889|
|INFORMATION AND C...|        19.427444794952706|
|        CONSTRUCTION|         20.94497041420119|
